# Model:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(X_train.shape, y_train.shape , X_test.shape , y_test.shape)

# Assuming y_train is integer labels
y_train_one_hot = to_categorical(y_train, num_classes=10)

# Reshape the input data to include the channel dimension
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))

# Define the model
model = Sequential()

# Add layers to the model
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dropout(0.25))

# Adjust the number of units in the output layer based on your classification task
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
es = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=4, verbose=1, restore_best_weights=True)
mc = ModelCheckpoint("model.h5", monitor="val_accuracy", verbose=1, save_best_only=True)

# Train the model without callbacks
history = model.fit(X_train, y_train_one_hot, epochs=50, validation_split=0.3)



(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)



Epoch 1/50


1313/1313 [==============================] - 11s 7ms/step - loss: 0.6987 - accuracy: 0.9050 - val_loss: 0.1065 - val_accuracy: 0.9687
Epoch 2/50
1313/1313 [==============================] - 10s 7ms/step - loss: 0.1198 - accuracy: 0.9650 - val_loss: 0.0784 - val_accuracy: 0.9766
Epoch 3/50
1313/1313 [==============================] - 10s 7ms/step - loss: 0.0910 - accuracy: 0.9727 - val_loss: 0.0903 - val_accuracy: 0.9752
Epoch 4/50
1313/1313 [==============================] - 10s 7ms/step - loss: 0.0829 - accuracy: 0.9749 - val_loss: 0.0578 - val_accuracy: 0.9822
Epoch 5/50
1313/1313 [==============================] - 10s 7ms/step - loss: 0.0746 - accuracy: 0.9776 - val_loss: 0.0632 - val_accuracy: 0.9815
Epoch 6/50
1313/1313 [==============================] - 10s 7ms/step - loss: 0.0677 - accuracy: 0.9788 - val_loss: 0.0624 - val_accuracy: 0.9813
Epoch 7/50
1313/1313 [==============================] - 10s 7ms/step - los

In [2]:
# Train the model with callbacks
history = model.fit(X_train, y_train_one_hot, epochs=50, validation_split=0.3, callbacks=[es, mc])

Epoch 1/50
1310/1313 [============================>.] - ETA: 0s - loss: 0.0392 - accuracy: 0.9948
Epoch 1: val_accuracy improved from -inf to 0.98539, saving model to model.h5
1313/1313 [==============================] - 10s 7ms/step - loss: 0.0392 - accuracy: 0.9949 - val_loss: 0.2018 - val_accuracy: 0.9854
Epoch 2/50
  25/1313 [..............................] - ETA: 8s - loss: 0.0177 - accuracy: 0.9962

C:\Users\tejus\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1305/1313 [============================>.] - ETA: 0s - loss: 0.0491 - accuracy: 0.9931
Epoch 2: val_accuracy improved from 0.98539 to 0.98683, saving model to model.h5
1313/1313 [==============================] - 10s 7ms/step - loss: 0.0488 - accuracy: 0.9931 - val_loss: 0.1695 - val_accuracy: 0.9868
Epoch 3/50
1306/1313 [============================>.] - ETA: 0s - loss: 0.0420 - accuracy: 0.9941
Epoch 3: val_accuracy improved from 0.98683 to 0.98883, saving model to model.h5
1313/1313 [==============================] - 10s 7ms/step - loss: 0.0423 - accuracy: 0.9941 - val_loss: 0.1483 - val_accuracy: 0.9888
Epoch 4/50
1305/1313 [============================>.] - ETA: 0s - loss: 0.0317 - accuracy: 0.9945
Epoch 4: val_accuracy did not improve from 0.98883
1313/1313 [==============================] - 10s 7ms/step - loss: 0.0318 - accuracy: 0.9945 - val_loss: 0.1386 - val_accuracy: 0.9885
Epoch 5/50
1311/1313 [============================>.] - ETA: 0s - loss: 0.0367 - accuracy: 0.9946Resto

# Application:

In [3]:
import pygame
import sys
import numpy as np
from tensorflow.keras.models import load_model
import cv2
# Press "N" to clear screen
WINDOWSIZEX = 640
WINDOWSIZEY = 480

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)

MODEL = load_model("model.h5")

LABELS = {0: "Zero", 1: "One",
          2: "Two", 3: "Three",
          4: "Four", 5: "Five",
          6: "Six", 7: "Seven",
          8: "Eight", 9: "Nine"}

pygame.init()

FONT = pygame.font.Font("freesansbold.ttf", 18)
DISPLAYSURF = pygame.display.set_mode((WINDOWSIZEX, WINDOWSIZEY))

pygame.display.set_caption("Digit Board")

iswriting = False
image_cnt = 1
BOUNDARYINC = 5
PREDICT = True
IMAGESAVE = False

number_xcord = []
number_ycord = []
rect_min_x, rect_min_y, rect_max_x, rect_max_y = 0, 0, 0, 0  # Initialize rectangle coordinates

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        if event.type == pygame.MOUSEMOTION and iswriting:
            xcord, ycord = event.pos
            pygame.draw.circle(DISPLAYSURF, WHITE, (xcord, ycord), 4, 0)

            number_xcord.append(xcord)
            number_ycord.append(ycord)

        if event.type == pygame.MOUSEBUTTONDOWN:
            iswriting = True

        if event.type == pygame.MOUSEBUTTONUP:
            iswriting = False
            if number_xcord and number_ycord:
                rect_min_x, rect_max_x = min(number_xcord) - BOUNDARYINC, max(number_xcord) + BOUNDARYINC
                rect_min_y, rect_max_y = min(number_ycord) - BOUNDARYINC, max(number_ycord) + BOUNDARYINC

                number_xcord = []
                number_ycord = []

                image_arr = np.array(pygame.PixelArray(DISPLAYSURF))[rect_min_x:rect_max_x, rect_min_y:rect_max_y].T.astype(np.float32)

                if IMAGESAVE:
                    cv2.imwrite(f"image_{image_cnt}.png", image_arr)
                    image_cnt += 1

                if PREDICT:
                    image = cv2.resize(image_arr, (28, 28))
                    image = np.pad(image, ((10, 10), (10, 10)), 'constant', constant_values=0)
                    image = cv2.resize(image, (28, 28)) / 255.0

                    label = str(LABELS[np.argmax(MODEL.predict(image.reshape(1, 28, 28, 1)))])
                    text_surface = FONT.render(label, True, RED, WHITE)
                    text_rect_obj = text_surface.get_rect()
                    text_rect_obj.left, text_rect_obj.bottom = rect_max_x + 5, rect_min_y

                    DISPLAYSURF.blit(text_surface, text_rect_obj)

                # Draw a rectangular box around the written number
                pygame.draw.rect(DISPLAYSURF, RED, (rect_min_x, rect_min_y, rect_max_x - rect_min_x, rect_max_y - rect_min_y), 2)

        if event.type == pygame.KEYDOWN:
            if event.unicode == 'n':
                DISPLAYSURF.fill(BLACK)

    pygame.display.update()


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
1/1 [==============================] - 0s 16ms/step


SystemExit: 

C:\Users\tejus\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
